In [ ]:
# | default_exp server

In [ ]:
# | export

import importlib
import sys
import asyncio
from typing import *
from contextlib import contextmanager
from pathlib import Path
import threading
import signal

import multiprocessing

from fastkafka.application import FastKafka
from fastkafka.testing import change_dir
from fastkafka._components.helpers import generate_app_src, _import_from_string

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory
from time import sleep

In [ ]:
# | export

class FastKafkaServer():
    
    def __init__(self, app: FastKafka):
        self.app = app
        self.should_exit = False
    
    
    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

#         async with self.app:
        await self.app.startup()
        await self._main_loop()
        await self.app.shutdown()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            # Signals can only be listened to from the main thread.
            return

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )
        
        def handle_exit(sig: int) -> None:
            self.should_exit = True
        
        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)
            
            
    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        import_str = f"{src_path.stem}:kafka_app"
        app = _import_from_string(import_str)
        async with app:
            sleep(1)
        # Patch _main_loop to last x seconds, patch fastkafka 
        # startup and shutdown and assert startup and shutdown called

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpymzjpnnx/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpymzjpnnx/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'group_id': 'tvrtko-fastkafka-kafka-1:9092_group', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'group_id': 'tvrtko-fastkafka-kafka-1:9092_group', 'auto_offset_reset': 'earliest', 'max_pol

In [ ]:
#| export

@contextmanager
def run_fastkafka_server(app: FastKafka) -> Generator[None, None, None]:
    app = app
    
    def run(app=app):
        server = FastKafkaServer(app=app)
        server.run()
        
    p = multiprocessing.Process(target=run)
    try:
        p.start()
        yield
    except Exception as e:
        print(f"Exception raised {e=}")
    finally:
        p.terminate()
        p.join()
        p.close()


In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        import_str = f"{src_path.stem}:kafka_app"
        app = _import_from_string(import_str)
        with run_fastkafka_server(app), run_fastkafka_server(app):
            sleep(10)

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp9tmixebw/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp9tmixebw/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp9tmixebw/asyncapi/spec/asyncapi.yml'


Process Process-3:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_881/1447747746.py", line 9, in run
    server.run()
  File "/tmp/ipykernel_881/378879156.py", line 11, in run
    return asyncio.run(self._serve())
  File "/usr/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 646, in run_until_complete
    return future.result()
  File "/tmp/ipykernel_881/378879156.py", line 17, in _serve
    await self.app.startup()
  File "/work/fastkafka/fastkafka/application.py", line 1395, in startup
    await self._populate_producers()
  File "/work/fastkafka/fastkafka/application.py", line 1327, in _populate_producers
    self._producers_store = {
  File "/work/fastkafka/fa

[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/tmp/tmp9tmixebw/asyncapi/spec/asyncapi.yml'


Process Process-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_881/1447747746.py", line 9, in run
    server.run()
  File "/tmp/ipykernel_881/378879156.py", line 11, in run
    return asyncio.run(self._serve())
  File "/usr/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 646, in run_until_complete
    return future.result()
  File "/tmp/ipykernel_881/378879156.py", line 17, in _serve
    await self.app.startup()
  File "/work/fastkafka/fastkafka/application.py", line 1395, in startup
    await self._populate_producers()
  File "/work/fastkafka/fastkafka/application.py", line 1327, in _populate_producers
    self._producers_store = {
  File "/work/fastkafka/fa